# A toy example
# Convert to Julia file by jupytext --to jl ToyExample.ipynb

In [ ]:
include("../src/DSI.jl")
include("../src/Zono_utils.jl")
include("../src/PZono.jl")
include("../src/DSZ.jl")

In [ ]:
print_figures = false;

In [ ]:
# Toy example: defining the network
W1 = [1 -1.0; 1.0 1.]
b = [0.0; 0.0]
W2 = [1 -1.0; 1.0 1.]
L1 = Layer(W1, b, ReLU())
L2 = Layer(W2, b, Id())
full_net = Network([L1; L2])

# input range
x = [interval(-2,2),interval(-1.,1.)]

In [ ]:
# Running the example with 2 focal elements
#nb_discretization_steps = 2
#ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
#px = uniform.(inf.(x),sup.(x)) 
# Printing the pbox
#pz = pbox_approximate_nnet(full_net,px,true) # independent inputs
#if print_figures
#    ProbabilityBoundsAnalysis.plot(px[1])
#    ProbabilityBoundsAnalysis.plot(pz[1])
#    ProbabilityBoundsAnalysis.plot(pz[2])
#end

In [ ]:
function Compare_analyzes(px::Vector{pbox})

    println("DSI Analysis, unknown correlation on inputs")
    @time dsi_pz = pbox_approximate_nnet(full_net,px,false)
    print("dsi_pz[1] <= -2 : ",dsi_pz[1] <= -2,"\n")
    print("dsi_pz[2] >= 2 : ",dsi_pz[2] >= 2,"\n")


    println("DSI analysis, independent inputs")
    @time dsi_pz = pbox_approximate_nnet(full_net,px,true)
    print("dsi_pz[1] <= -2 : ",dsi_pz[1] <= -2,"\n")
    print("dsi_pz[2] >= 2 : ",dsi_pz[2] >= 2,"\n")


    println("Probabilistic zonotopes analysis, unknown correlation on inputs")
    @time pzono_pz = affpbox_approximate_nnet(full_net,px,false)
    print("pzono_pz[1] <= -2 : ",pzono_pz[1] <= -2,"\n")
    print("pzono_pz[2] >= 2 : ",pzono_pz[2] >= 2,"\n")

    
    println("Probabilistic zonotope analysis, independent inputs")
    @time pzono_pz = affpbox_approximate_nnet(full_net,px,true) # true for independent inputs
    print("pzono_pz[1] <= -2 : ",pzono_pz[1] <= -2,"\n")
    print("pzono_pz[2] >= 2 : ",pzono_pz[2] >= 2,"\n")

    #println("Zonotopic Dempster-Shafer analysis, independent inputs")
    #@time begin
    #    dsz = generate_DSZ(px)
    #    dsz_res = dsz_approximate_nnet(full_net, dsz)
    #    dsztodsi = DSZ_to_pbox(dsz_res,true)
    #end
    #print("dszpz[1] <= -2 : ",dsztodsi[1] <= -2,"\n")
    #print("dszpz[1] >= 2 : ",dsztodsi[2] >= 2,"\n")

    println("Zonotopic Dempster-Shafer analysis, independent inputs")
    mat_spec_1 = [[1.0 0.0]
                  [0.0 -1.0]]
    rhs_spec_1 = [-2.0, -2.0]
    #@time dsz_approximate_nnet_and_condition(full_net, px, mat_spec_1,rhs_spec_1,true) 
    println("Property conjunction")
    @time dsz_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_1,rhs_spec_1,true) 

    #mat_spec_1stcondition = [1.0 0.0]
    ##rhs_spec_1stcondition = [-2.0]
    #mat_spec_2ndcondition = [0.0 -1.0]
    #rhs_spec_2ndcondition = [-2.0]
    #println("1st property")
    #@time dsz_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_1stcondition,rhs_spec_1stcondition,true) 
    #println("2nd property")
    #@time dsz_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_2ndcondition,rhs_spec_2ndcondition,true) 
end

In [ ]:
#ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
#px = [(normal(interval(0,1),1)-0.5)/3.5902*2.,(normal(interval(0,1),1)/2.0-0.25)/1.7951]

for nb_discretization_steps in (2,10,100, 1000)
    println("Number of focal elements on the inputs = ",nb_discretization_steps)
    ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
    println("Uniform law")
    px = uniform.(inf.(x),sup.(x)) 
    Compare_analyzes(px)

    println("********************")
    println("Truncated Normal law")
    px = init_pbox_Normal(inf.(x),sup.(x),nb_discretization_steps,true) # truncated normal law
    Compare_analyzes(px)
    println("************************************************************************************************************************")
    println("************************************************************************************************************************")
end